# INFO 4271 - Group Project

Issued: June 17, 2025

Due: July 21, 2025

Please submit a link to your code repository (with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [ ]:
%load_ext autoreload
%autoreload 2

from project import Crawler
import cProfile

start_urls = []
with open('seed.txt', 'r') as file:
    for line in file: start_urls.append(line.strip())
start_urls
crawler = Crawler(urls=start_urls, max_workers=200, use_proxies=False)
# snakeviz profiler.prof
cProfile.run('crawler.run()', 'data/profiler.prof')

In [3]:
filtered = {p for p, d in crawler.visited_pages.items() if d["keywords_found"] and d["is_english"]}
print(len(filtered), len(crawler.visited_pages))
# filtered

10365 34448


In [ ]:
display(list(crawler.urls_to_visit)[:100])

['https://sinzlab.org/projects/pose.html',
 'http://www.encyclopedia.com/social-sciences/dictionaries-thesauruses-pictures-and-press-releases/gouldner-alvin-w',
 'http://www.wikidata.org/wiki/Q12027957',
 'https://xtools.wmflabs.org/topedits/commons.wikimedia.org/Sanbec/0/T%C3%BCbingen?uselang=tg-cyrl',
 'https://en.wikipedia.org/wiki/ISO_3166-2:DE',
 'https://necludov.github.io/',
 'https://historicgermany.travel/events/list/page/2/?tribe-bar-date=2025-07-11',
 'https://uni-tuebingen.de/en/fakultaeten/philosophische-fakultaet/fachbereiche/geschichtswissenschaft/seminareinstitute/geschichtsdidaktik-und-public-history/personen/buschhaus/vita/',
 'https://en.wikipedia.org/wiki/Template_talk:OSCE',
 'https://www.wikidata.org/wiki/Q30',
 'http://www.uni-tuebingen.de/en/study/prospective-students/tuebingen-as-a-place-to-study/',
 'https://www.nature.com/articles/s41467-024-51546-9?fromPaywallRec=true',
 'https://github.com/wichmann-lab/trivial-or-impossible/blob/main/04_figure_4.ipynb',
 'h

In [18]:
from urllib.parse import urlparse
from collections import Counter

c = 0
c2 = 0
counter = Counter()
for url in crawler.urls_to_visit:
    t = url in crawler.visited_pages.keys()
    c += t
    domain = crawler.get_domain(urlparse(url))
    c2 +=  crawler.domain_dict[domain]["in_use"]
    counter[domain] += 1
    # if t: print(url)
print(c, c2)
print(counter)
print(len(counter))
print(counter.total())

2945 0
Counter({'google.com': 5167, 'encyclopedia.com': 4443, 'uni-tuebingen.de': 4354, 'ebdestinations.com': 4169, 'github.com': 3533, 'wmflabs.org': 3064, 'wanderlog.com': 2545, 'doi.org': 2242, 'historicgermany.travel': 1949, 'wikipedia.org': 1883, 'europeanbestdestinations.com': 1339, 'tuebingenresearchcampus.com': 1285, 'arxiv.org': 1157, 'mpg.de': 1070, 'wikidata.org': 866, 'github.io': 844, 'wikimedia.org': 803, 'dzif.de': 729, 'mloss.org': 616, 'co.uk': 508, 'youtube.com': 408, 'cyber-valley.de': 385, 'britannica.com': 381, 'nature.com': 352, 'uni-stuttgart.de': 342, 'wikivoyage.org': 337, 'studit-tuebingen.de': 335, 'hih-tuebingen.de': 335, 'civis.eu': 313, 'curevac.com': 257, 'npr.org': 227, 'frontiersin.org': 208, 'clarivate.com': 178, 'nexteinstein.org': 169, 'openreview.net': 169, 'bethgelab.org': 149, 'artist-info.com': 149, 'jneurosci.org': 135, 'biomedcentral.com': 134, 'igjr.org': 129, 'inaturalist.org': 123, 'my-stuwe.de': 109, 'plos.org': 108, 'medrxiv.org': 104, 'jm

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file (see `queries.txt` for an example) will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
#Retrieve documents relevnt to a query. You need (at least) two parameters:
	#query: The user's search query
	#index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me
	pass

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 22nd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
